In [1]:
!git clone https://github.com/supervisely/developer-portal

Cloning into 'developer-portal'...
remote: Enumerating objects: 5258, done.
remote: Counting objects: 100% (1774/1774), done.
remote: Compressing objects: 100% (726/726), done.
remote: Total 5258 (delta 1202), reused 1416 (delta 1021), pack-reused 3484
Receiving objects: 100% (5258/5258), 79.68 MiB | 14.07 MiB/s, done.
Resolving deltas: 100% (3385/3385), done.


In [1]:
from dotenv import load_dotenv
load_dotenv(".env")

True

In [3]:
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

In [2]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

docs_path = "developer-portal"

loader = DirectoryLoader(docs_path, glob="**/*.md", loader_cls=TextLoader)

docs = loader.load()
len(docs)

249

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
len(splits)

482

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma

embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-base",
    model_kwargs={"device": 0},  # Comment out to use CPU
)

vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory="./supervisely-dev-portal-db",
)

vectorstore.add_documents(splits)

In [28]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [40]:
vectorstore.similarity_search_with_relevance_scores(q)

[(Document(page_content='**Optional fields and loading** These fields are optional and are not needed when loading the project. The server can automatically fill in these fields while project is loading.\n\n* `id` - unique identifier of the current object\n* `classId` - unique class identifier of the current object\n* `labelerLogin` - string - the name of user who created the current figure\n* `createdAt` - string - date and time of figure creation\n* `updatedAt` - string - date and time of the last figure update\n\nMain idea of `key` fields and `id` you can see below in [Key id map file](point-clouds.md#key-id-map-file) section.\n\n**Fields definitions:**\n\n* `description` - string - (optional) - this field is used to store the text to assign to the sequence.\n* `key` - string, unique key for a given sequence (used in key\\_id\\_map.json to get the sequence ID)\n* `tags` - list of strings that will be interpreted as point cloud tags\n* `objects` - list of objects that may be present 

In [37]:
retriever.invoke(q)

[Document(page_content='**Optional fields and loading** These fields are optional and are not needed when loading the project. The server can automatically fill in these fields while project is loading.\n\n* `id` - unique identifier of the current object\n* `classId` - unique class identifier of the current object\n* `labelerLogin` - string - the name of user who created the current figure\n* `createdAt` - string - date and time of figure creation\n* `updatedAt` - string - date and time of the last figure update\n\nMain idea of `key` fields and `id` you can see below in [Key id map file](point-clouds.md#key-id-map-file) section.\n\n**Fields definitions:**\n\n* `description` - string - (optional) - this field is used to store the text to assign to the sequence.\n* `key` - string, unique key for a given sequence (used in key\\_id\\_map.json to get the sequence ID)\n* `tags` - list of strings that will be interpreted as point cloud tags\n* `objects` - list of objects that may be present o

In [26]:
from operator import itemgetter
from typing import Sequence

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.document import Document
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable.passthrough import RunnableAssign


# After the retriever fetches documents, this
# function formats them in a string to present for the LLM
def format_docs(docs: Sequence[Document]) -> str:
    formatted_docs = []
    for i, doc in enumerate(docs):
        doc_string = (
            f"<document index='{i}'>\n"
            f"<source>{doc.metadata.get('source')}</source>\n"
            f"<doc_content>{doc.page_content}</doc_content>\n"
            "</document>"
        )
        formatted_docs.append(doc_string)
    formatted_str = "\n".join(formatted_docs)
    return f"<documents>\n{formatted_str}\n</documents>"


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Here are some documents from Supervisely Developer docs to expand your knowledge about the topic:"
            "\n{context}\n"
            "Carefully respond to the user query using the provided documents. It there is no relevant information, mention it to the user and try to answer based on your knowledge.",
        ),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(temperature=0.6, model="gpt-3.5-turbo-1106")

response_generator = (prompt | llm | StrOutputParser()).with_config(
    run_name="GenerateResponse",
)

# This is the final response chain.
# It fetches the "question" key from the input dict,
# passes it to the retriever, then formats as a string.

chain = (
    RunnableAssign(
        {
            "context": (itemgetter("question") | retriever | format_docs).with_config(
                run_name="FormatDocs"
            )
        }
    )
    # The "RunnableAssign" above returns a dict with keys
    # question (from the original input) and
    # context: the string-formatted docs.
    # This is passed to the response_generator above
    | response_generator
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [42]:
trace = []
for step in chain.stream({"question": q}):
    trace.append(step)

In [38]:
q = '''When I load a Point Cloud ".pcd" file, what will be x,y,z and other dimensions mean?'''
r = chain.invoke({"question": q})
print(r)

When loading a Point Cloud ".pcd" file, the x, y, and z dimensions represent the spatial coordinates of each point in the point cloud. These coordinates define the position of the point in 3D space. The r, g, and b dimensions represent the color information for each point, with values ranging from 0 to 255 for the red, green, and blue color channels, respectively.

Here's a breakdown of the dimensions:
- **x**: The x coordinate of the point
- **y**: The y coordinate of the point
- **z**: The z coordinate of the point
- **r**: The red color channel component (0-255)
- **g**: The green color channel component (0-255)
- **b**: The blue color channel component (0-255)

These values collectively define the spatial position and color of each point in the point cloud.


In [39]:
print(r, file=open("output.txt", "w"))

In [83]:
[x.metadata for x in vectorstore.similarity_search(q, k=5)]

[{'source': 'developer-portal/api-references/supervisely-annotation-json-format/point-clouds.md'},
 {'source': 'developer-portal/api-references/supervisely-annotation-json-format/point-clouds.md'},
 {'source': 'developer-portal/getting-started/python-sdk-tutorials/point-clouds/point-clouds-and-episodes.md'},
 {'source': 'developer-portal/api-references/supervisely-annotation-json-format/volumes-annotation.md'},
 {'source': 'developer-portal/api-references/supervisely-annotation-json-format/point-cloud-episodes.md'}]

In [86]:
retriever.invoke(q)[1].page_content

'# Point Clouds\n\n<figure><img src="https://github.com/supervisely/developer-portal/raw/main/.gitbook/assets/3d_pointclouds_interface.png" alt=""><figcaption><p>3D Point Cloud labeling interface</p></figcaption></figure>\n\n## Project Structure Example\n\n```\n<PROJECT_NAME>  \n├── key_id_map.json (optional)              \n├── meta.json     \n├── <DATASET_NAME_1>                         \n│ ├── pointcloud                    \n│ │   ├── scene_1.pcd           \n│ │   ├── scene_2.pcd   \n│ │   └── ...                \n│ ├── related_images (optional)               \n│ │   ├── scene_1_pcd               \n│ │   │ ├── scene_1_cam0.png       \n│ │   │ ├── scene_1_cam0.png.json  \n│ │   │ ├── scene_1_cam1.png       \n│ │   │ ├── scene_1_cam1.png.json  \n│ │   │ └── ... \n│ │   ├── scene_2_pcd               \n│ │   │ ├── scene_2_cam0.png       \n│ │   │ ├── scene_2_cam0.png.json  \n│ │   │ ├── scene_2_cam1.png       \n│ │   │ ├── scene_2_cam1.png.json  \n│ │   │ └── ... \n│ │   └── ...      \n│

In [ ]:
[x.metadata for x in vectorstore.similarity_search(q, k=5)]

In [10]:
prompt = hub.pull("rlm/rag-prompt")


In [35]:
x = "autonomous agents"
r = retriever.invoke(x)

In [36]:
print(r[0].page_content)

They also discussed the risks, especially with illicit drugs and bioweapons. They developed a test set containing a list of known chemical weapon agents and asked the agent to synthesize them. 4 out of 11 requests (36%) were accepted to obtain a synthesis solution and the agent attempted to consult documentation to execute the procedure. 7 out of 11 were rejected and among these 7 rejected cases, 5 happened after a Web search while 2 were rejected based on prompt only.
Generative Agents Simulation#
Generative Agents (Park, et al. 2023) is super fun experiment where 25 virtual characters, each controlled by a LLM-powered agent, are living and interacting in a sandbox environment, inspired by The Sims. Generative agents create believable simulacra of human behavior for interactive applications.
The design of generative agents combines LLM with memory, planning and reflection mechanisms to enable agents to behave conditioned on past experience, as well as to interact with other agents.


In [ ]:
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Task Decomposition?")